In [ ]:
push!(LOAD_PATH, ".")
using PyPlot
using LujiaLt
import Fig3DTools

In [ ]:
# BCC unit cell
X = [ [0,0,0] [1,0,0] [0,1,0] [1,1,0]  [0,0,1] [1,0,1] [0,1,1] [1,1,1] [.5,.5,.5] ]
Fig3DTools.write_xyz(X, "bcc_unit_cell.xyz", comment="bcc unit cell")

# FCC unit cell
Xfcc = [ [0,0,0] [1,0,0] [0,1,0] [1,1,0] [0,0,1] [1,0,1] [0,1,1] [1,1,1] [.5,.5,0] [.5,.5,1] [.5, 0, .5] [.5, 1, .5] [0, .5, .5] [1,.5,.5] ]
Fig3DTools.write_xyz(X, "fcc_unit_cell.xyz", comment="fcc unit cell")

# hexagonal lattice
X = Fig3DTools.multilattice_ball(R=10)
Fig3DTools.write_xyz(X, "hex_lattice.xyz", comment="hexagonal lattice unit cell")

# Diamond cubic
X_diamond = [Xfcc ([.25,.25,.25] .+ Xfcc)]
Fig3DTools.write_xyz(X_diamond, "diamond_cubic.xyz", comment="diamong cubic unit cell")
X_diamond_cell = X_diamond[:, find(maximum(X_diamond, 1) .<= 1.01)]
Fig3DTools.write_xyz(X_diamond_cell, "diamond_cubic_cell.xyz", comment="diamong cubic unit cell")


In [ ]:
# generate a vacancy geometry
X = Fig3DTools.lattice_ball(R=20.1, lattice = "tri", defect = "vac")
# call the solver, with free boundary
Y = LujiaLt.quick_solve(X=X, Rfree=18.1, show_trace=false)
# PyPlot points
plot(Y[1,:][:], Y[2,:][:], "b.")
# write xyz file
Fig3DTools.write_xyz(X, "tri_vacancy.xyz");

In [ ]:
# generate a interstitial geometry
X = Fig3DTools.lattice_ball(R=15, lattice = "tri", defect = "int")
# call the solver, with free boundary
Y = copy(X)
Y = LujiaLt.quick_solve(X=Y, Rfree = 12.1, show_trace=false)
# PyPlot points
plot(Y[1,:], Y[2,:], "b.")
# write xyz file
LjIO.write_xyz(Y, "tri_interstial.xyz");

In [ ]:
# displacement field for an edge dislocation
function u_edge(X, b, ν)
    # shift the lattice
    X1 = X[1,:]
    X2 = X[2,:]
    # compute the displacements
    # note the extra shift u1 += b/2 is to take into account discreteness
    X1[find(X2 .< 0)] += b/2
    u1 = b/(2*π)*( atan(X1 ./ X2) + (X1 .* X2) ./ (2*(1-ν)*(X1.^2+X2.^2)) )
    u2 = -b/(2*π)* ( (1-2*ν)/(4*(1-ν)) * log(X1.^2 + X2.^2) + (X1.^2-X2.^2) ./ (4*(1-ν)*(X1.^2+X2.^2)) )
    # apply the displacement to the lattice
    X1 += u1
    X2 += u2
    # return deformed lattice
    return [X1; X2]
end

In [ ]:

####### EDGE DISLOCATION in Z2 ########
N = 20
s = -N:N
o = zeros(2*N+1)
x = s .+ o'; y = o .+ s'
X = [x[:]'; y[:]']
# shift a cell-centre into the origin
X = X .- [.5; .5]

# save this (???why???)
Fig3DTools.write_xyz(X, "Z2.xyz")

# apply isotropic linear elasticity solution
X = u_edge(X, 1.0, 1/4)

# assign different types to a few of the particles
Icore = find( (X[1,:] .> -1) & (X[1,:] .< 1) & (X[2,:] .> 0) & (X[2,:] .< 2) )
S = [ "Fe" for n = 1:size(X, 2) ]
S[Icore] = "Al"

# plot the LE configuration
plot(X[1,:][:], X[2,:][:], "b.", markersize=10)
plot(X[1,Icore][:], X[2,Icore][:], "r*")
axis([-4,4,-3,5])
# and output as XYZ
Fig3DTools.write_xyz(X, "Z2_edgedis.xyz"; species=S)

# create a periodically repeated image
Y = reshape(Float64[], 3, 0)
S2 = []
o = ones(size(X, 2))'
for n = 0:5
    Y = [Y [X; n*o]]
    S2 = [S2; S]
end
Fig3DTools.write_xyz(Y, "Z3_edgedis.xyz"; species=S2)


In [ ]:
####### EDGE DISLOCATION IN TRI LATTICE ########
X, I0 = Fig3DTools.lattice_ball(R=20.1, lattice = "tri", defect = "")
# scale the by to get nn-distance right
nn_dist = 0.99    # (WHY!?!?!?)
X *= nn_dist
# shift the lattice so the core is in the origin
X = X .- (X[:,I0] + [0.5, sqrt(3)/4])
# apply isotropic linear elasticity solution
X = u_edge(X, nn_dist, 1/4)
# slightly reduce the computational domain
iIn = find(Fig3DTools.get_r(X) .< 18.1)
X = X[:, iIn]
# freeze boundary region
iFree = find(Fig3DTools.get_r(X) .< 14.1)

# plot reference configuration
# plot(X[1,:][:], X[2,:][:], "k.", markersize=5)
# plot(X[1,iFree][:], X[2,iFree][:], "b.", markersize=10)

# relax the LE guess
Y = LujiaLt.quick_solve(X=X, Ifree=iFree, show_trace=false)

# # and output as XYZ
Fig3DTools.write_xyz(Y, "tri_edgedis.xyz")

# and/or plot here in the notebook
plot(Y[1,:][:], Y[2,:][:], "k.", markersize=5)
plot(Y[1,iFree][:], Y[2,iFree][:], "b.", markersize=10);

In [ ]:
# Screw Dislocation in Z3
N = 3
X = Fig3DTools.dgrid_list(-N+1:N, 3)
X = X .- [.5;.5;.5]
# assign to each plane a separate species
s = ["Fe", "Al"]
S = ["" for n = 1:size(X,2)]
for n = -N+1:N
    z = n - 0.5
    Iz = find( abs(X[3, :] - z) .< 0.1 )
    S[Iz] = s[mod(n, 2)+1]
end
# write Z3    
Fig3DTools.write_xyz(X, "Z3.xyz"; species=S)
# write screw
u = angle(X[1,:] + im * X[2,:]) / (2*pi)
X[3,:] += u
Fig3DTools.write_xyz(X, "Z3_screw.xyz"; species = S)


In [ ]:
# STONE-WALES DEFECT
X = Fig3DTools.multilattice_ball(R=20)
I1 = find(sum(abs(X), 1) .<= 0.1)
I2 = find( (abs(X[1,:]) .<= 0.1) & (abs(X[2,:]-1.) .<= 0.1) )
I = [I1; I2]
c = 0.5 * (X[:, I1] + X[:, I2])
X = X .- c
X[:, I] = [0.8*X[2, I]; - X[1, I]]

plot(X[1,:][:], X[2,:][:], "b.")
plot(X[1, I][:], X[2,I][:], "rs")

Fig3DTools.write_xyz(X, "stone_wales_lge.xyz", comment="hexagonal lattice, stone wales defect, not equilibrated")



### Visualisation of the screw dislocation definition



In [ ]:
# embedded Z2
N = 10
t = linspace(-N, N, 2*N+1)
o = ones(2*N+1)
x, y, z = t*o', o*t', 0.*o*o'
Y = [x[:]';  y[:]';  z[:]']
S = ["Fe" for n = 1:size(Y,2)]

# pillar 1
Y = [Y [0*t'; 0*t'; 0.2+t']]
al = ["Al" for n = 1:N]
S = [S; al; "N"; al]
# pillar 2
Y = [Y [1.+0*t'; 0*t'; 0.8+t']]
S = [S; al; "N"; al]
# pillar 3
Y = [Y [-1.+0*t'; 0*t'; 0.6+t']]
S = [S; al; "N"; al]

Fig3DTools.write_xyz(Y, "screw_defn.xyz"; species=S)


In [ ]:
# embedded Z2
N = 10
t = linspace(-N, N, 2*N+1)
o = ones(2*N+1)
x, y, z = t*o', o*t', 0.*o*o'
Y = [x[:]';  y[:]';  z[:]']
S = ["Fe" for n = 1:size(Y,2)]

# pillar 1
Y = [Y [0*t'; 0*t'; 0.0+t']]
al = ["Al" for n = 1:N]
S = [S; al; "N"; al]
# pillar 2
Y = [Y [1.+0*t'; 0*t'; 0.25+t']]
S = [S; al; "N"; al]
# pillar 3
Y = [Y [1.+0*t'; 1.+0.*t'; 0.5+t']]
S = [S; al; "N"; al]
# pillar 4
Y = [Y [0*t'; 1.+0*t'; 0.75+t']]
S = [S; al; "N"; al]

Fig3DTools.write_xyz(Y, "screw_burg_cell.xyz"; species=S)

